## Prelims

In [1]:
# Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
%load_ext autoreload
%autoreload 2

### Install dependencies

In [2]:
# This only needs to be run ONCE:
# !pip install -qU magenta pyfluidsynth pretty_midi

### Import packages

In [3]:
import random
from dataclasses import dataclass

import magenta
import note_seq
import numpy as np
from note_seq import midi_synth
from note_seq.protobuf import music_pb2
from note_seq.sequences_lib import stretch_note_sequence

import m00sic
from m00sic.core import Note
from m00sic.core import NOTES_PER_OCTAVE
from m00sic.core import MajorKey
from m00sic.core import MinorKey
from m00sic.core import PitchClass
from m00sic.core import VALID_PITCH_CLASSES
from m00sic.core import WHOLE, W
from m00sic.core import HALF, H
from m00sic.core import QUARTER, Q
from m00sic.core import EIGHTH, E
from m00sic.core import SIXTEENTH, S
from m00sic.utils import NoteSpec
from m00sic.utils import PatternSpec
from m00sic.utils import arrange_chord
from m00sic.utils import arrange_melody
from m00sic.utils import concat_sequences
from m00sic.utils import stack_sequences
from m00sic.utils import AFTER
from m00sic.utils import PAUSE

In [4]:
print(f"Magenta version: {magenta.__version__}")

Magenta version: 2.1.3


```python
@dataclasses.dataclass
class PatternSpec:
    index: int
    start_time: float
    duration: float
    transpose: int = 0
    velocity: int = 80
```

```python
@dataclasses.dataclass
class NoteSpec:
    """A specification for how a specific note should be played."""
    note: Note
    duration: float
    start_margin: float = 0.0
    end_margin: float = 0.0
    transpose: int = 0
    velocity: int = 80
```

In [5]:
PS = PatternSpec
NS = NoteSpec

## Cantus firmus

http://openmusictheory.com/cantusFirmus.html

In [101]:
KEY = MajorKey("C")

# Length between 8 and 16 notes.
LENGTH = 12

# Arythmic -- all whole notes.
rhythm = [1] * LENGTH

# Climax.
CLIMAX_IDX = 6
CLIMAX_DEGREE = 7

Other constraints:
- Range no more than a tenth, usually less than an octave.
- All note-to-note progressions are melodic consonances.
- Mostly stepwise motion, but some leaps.
- No more than 2 leaps in a row.
- No consecutive leaps in the same direction.
- Any large leaps (fourth or larger) are followed by step in opposite direction.
- The leading tone progresses to the tonic

In [102]:
def get_range(notes):
    not_none_notes = [note for note in notes if note is not None]
    return max(not_none_notes) - min(not_none_notes)


def only_contains_consonant_intervals(notes):
    for a, b in zip(notes[:-1], notes[1:]):
        # Check that it's not a seventh.
        if b - a == 6:
            return False
    # Since we're in a major key, all other intervals are consonant.
    return True


def contains_repetitions(notes):
    for a, b in zip(notes[:-1], notes[1:]):
        if a == b:
            return True
    return False


def max_consecutive_leaps(notes):
    max_leaps = 0
    num_leaps = 0
    for a, b in zip(notes[:-1], notes[1:]):
        if np.abs(b - a) > 1:
            num_leaps += 1
            max_leaps = max(max_leaps, num_leaps)
        else:
            num_leaps = 0
    return max_leaps


def contains_consecutive_leaps_in_same_dir(notes):
    for a, b, c in zip(notes[:-2], notes[1:-1], notes[2:]):
        two_leaps_up = (b - a > 1) and (c - b > 1)
        two_leaps_down = (b - a < -1) and (c - b < -1)
        if two_leaps_up or two_leaps_down:
            return True
    return False


def large_leaps_followed_by_reverse_step(notes):
    for a, b, c in zip(notes[:-2], notes[1:-1], notes[2:]):
        if b - a >= 3 and c - b != -1:
            return False
        if b - a <= -3 and c - b != 1:
            return False
    return True


def leading_tone_progresses_to_tonic(notes):
    for a, b in zip(notes[:-1], notes[1:]):
        if a == -1 and b != 0:
            return False
    return True


def approaches_final_tonic_by_step(notes):
    return notes[-2] == -1 or notes[-2] == 1


def has_no_leaps_bigger_than_fifth(notes):
    for a, b in zip(notes[:-1], notes[1:]):
        if abs(b - a) > 4:
            return False
    return True


def get_num_big_leaps(notes, big_leap_threshold=3):
    num_big_leaps = 0
    for a, b in zip(notes[:-1], notes[1:]):
        if abs(b - a) >= big_leap_threshold:
            num_big_leaps += 1
    return num_big_leaps


def is_valid_subcantus(seq):
    return (
        get_range(seq) < 8
        and only_contains_consonant_intervals(seq)
        and not contains_repetitions(seq)
        and max_consecutive_leaps(seq) <= 2
        and not contains_consecutive_leaps_in_same_dir(seq)
        and large_leaps_followed_by_reverse_step(seq)
        and leading_tone_progresses_to_tonic(seq)
        and has_no_leaps_bigger_than_fifth(seq)
        and get_num_big_leaps(seq) <= 1
    )

In [103]:
def try_to_fill_in_cantus(
    skeleton,
    missing_indices,
    allowed_min_degree,
    climax_degree,
):
    cantus = skeleton.copy()

    # Set second to last note.
    cantus[-2] = random.choice([-1, 1])
    
    # Iteratively fill in missing indices.
    for idx in missing_indices:
        candidates = list(range(allowed_min_degree, climax_degree))
        random.shuffle(candidates)
        success = False
        for candidate in candidates:
            cantus[idx] = candidate
            if is_valid_subcantus(cantus[:idx+1]):
                success = True
                break
        if not success:
            return False, None

    return is_valid_subcantus(cantus), cantus


def create_cantus_skeleton(length, climax_idx, climax_degree):
    skeleton = [None] * length
    skeleton[0] = 0
    skeleton[-1] = 0
    skeleton[climax_idx] = climax_degree
    return skeleton


def generate_cantus(length, climax_idx, climax_degree):
    skeleton = create_cantus_skeleton(length, climax_idx, climax_degree)
    missing_indices = [
        idx
        for idx, elem in enumerate(skeleton)
        if elem is None and idx != length - 2
    ]
    allowed_min_degree = climax_degree - 8

    success = False
    while not success:
        success, cantus_firmus = try_to_fill_in_cantus(
            skeleton,
            missing_indices,
            allowed_min_degree,
            climax_degree,
        )
    return cantus_firmus

In [104]:
cantus = generate_cantus(
    length=LENGTH,
    climax_idx=CLIMAX_IDX,
    climax_degree=CLIMAX_DEGREE,
)

In [105]:
print(f"Range: {get_range(cantus)} (< 8)")
print(f"Only consonant intervals: {only_contains_consonant_intervals(cantus)} (True)")
print(f"Contains repetitions: {contains_repetitions(cantus)} (False)")
print(f"Max consecutive leaps: {max_consecutive_leaps(cantus)} (<= 2)")
print(f"Contains consecutive leaps in same direction: {contains_consecutive_leaps_in_same_dir(cantus)} (False)")
print(f"Large leaps followed by reverse step: {large_leaps_followed_by_reverse_step(cantus)} (True)")
print(f"Leading tone progresses to tonic: {leading_tone_progresses_to_tonic(cantus)} (True)")
print(f"Has no leaps bigger than fifth: {has_no_leaps_bigger_than_fifth(cantus)} (True)")
print(f"Num big leaps: {get_num_big_leaps(cantus)} (<= 1)")

Range: 7 (< 8)
Only consonant intervals: True (True)
Contains repetitions: False (False)
Max consecutive leaps: 2 (<= 2)
Contains consecutive leaps in same direction: False (False)
Large leaps followed by reverse step: True (True)
Leading tone progresses to tonic: True (True)
Has no leaps bigger than fifth: True (True)
Num big leaps: 1 (<= 1)


In [106]:
melody = arrange_melody(
    degrees=cantus,
    rhythm=rhythm,
    octave=4,
    key=KEY,
)
melody_stretched = stretch_note_sequence(melody, 0.5)

note_seq.plot_sequence(melody)
note_seq.play_sequence(melody_stretched, synth=note_seq.fluidsynth)

Loading BokehJS ...

## First-species counterpoint

- Composing a second melody below a cantus.
- First and final note must both be either P1 or P8 below the cantus.
- Approach final interval by contrary stepwise motion (re-do for cantus -> ti-do for counterpoint).
- Counterpoint should have a single climax that doesn't coincide with the cantus's climax.
- A single repeat is allowed, but try to avoid it at all.
- Avoid voice crossing.
- Avoid voice overlap, where one voice leaps past the previous note of the other voice.
- Intervals should not exceed a perfect twelfth. In general, best to be within an octave. Only exceed a tenth in emergencies, for one or two notes.
- All harmonic consonances are allowed. Unisons only for first and last intervals.
- Imperfect consonances are preferable to perfect consonances for all intervals other than the first and last ones.
- Never, ever, ever use two perfect consonances of the same size in a row: P5-P5 or P8-P8, including compound intervals. In general, try to follow every perfect consonance with an imperfect consonance.
- Vary the types of motion between successive intervals (parallel, similar, contrary, oblique). Try to use all types of motion (except, perhaps, oblique motion), but prefer contrary motion where possible.
- Do not use more than three of the same imperfect consonance type in a row (e.g., three thirds in a row).
- Never move into a perfect consonance by similar motion (this is called direct or hidden octaves). This draws too much attention to an interval which already stands out of the texture.
- Avoid combining similar motion with leaps, especially large ones.

In [107]:
def num_repeats(notes):
    n_repeats = 0
    for a, b in zip(notes[:-1], notes[1:]):
        if a == b:
            n_repeats += 1
    return n_repeats


def voices_cross(cantus, first_species):
    for a, b in zip(cantus, first_species):
        if a + 7 < b:
            return True
    return False


def voices_overlap(cantus, first_species):
    for a, b in zip(cantus[:-1], first_species[1:]):
        if b > a + 7:
            return True
    for a, b in zip(cantus[1:], first_species[:-1]):
        if a < b - 7:
            return True
    return False


def intervals_exceed_perfect_twelfth(cantus, first_species, key):
    for a, b in zip(cantus, first_species):
        note_a = key.get_note(a, octave=4)
        note_b = key.get_note(b, octave=3)
        diff = note_a.midi_num - note_b.midi_num
        if diff > 19:
            return True
    return False


def num_intervals_over_tenth(cantus, first_species, key):
    counter = 0
    for a, b in zip(cantus, first_species):
        note_a = key.get_note(a, octave=4)
        note_b = key.get_note(b, octave=3)
        diff = note_a.midi_num - note_b.midi_num
        if diff > 16:
            counter += 1
    return counter


def only_consonances(cantus, first_species, key, final=False):
    subcantus = cantus[1:-1] if final else cantus[1:]
    sub_first_species = first_species[1:-1] if final else first_species[1:]
    for a, b in zip(subcantus, sub_first_species):
        note_a = key.get_note(a, octave=4)
        note_b = key.get_note(b, octave=3)
        diff = note_a.midi_num - note_b.midi_num
        if diff not in [3, 4, 7, 8, 9, 12, 15, 16, 19]:
            return False
    return True


def num_perfect_consonances(cantus, first_species, key):
    counter = 0
    for a, b in zip(cantus, first_species):
        note_a = key.get_note(a, octave=4)
        note_b = key.get_note(b, octave=3)
        diff = note_a.midi_num - note_b.midi_num
        if diff in [0, 7, 12, 19]:
            counter += 1
    return counter


def perfect_followed_by_imperfect(cantus, first_species, key):
    for a1, a2, b1, b2 in zip(cantus[:-1], cantus[1:], first_species[:-1], first_species[1:]):
        interval_1 = key.get_note(a1, octave=4).midi_num - key.get_note(b1, octave=3).midi_num
        interval_2 = key.get_note(a2, octave=4).midi_num - key.get_note(b2, octave=3).midi_num
        if interval_1 in [0, 7, 12, 19] and interval_2 in [0, 7, 12, 19]:
            return False
    return True


def more_than_three_imperfects_in_a_row(cantus, first_species):
    last_interval = None
    counter = 0
    for a, b in zip(cantus, first_species):
        interval = a + 7 - b
        if interval == last_interval:
            counter += 1
        else:
            counter = 0
            last_interval = interval
        if counter > 3:
            return True
    return False


def moves_into_perfect_by_similar_motion(cantus, first_species, key):
    for a1, a2, b1, b2 in zip(cantus[:-1], cantus[1:], first_species[:-1], first_species[1:]):
        similar_motion = (a1 < a2 and b1 < b2) or (a1 > a2 and b1 > b2)
        interval_2 = key.get_note(a2, octave=4).midi_num - key.get_note(b2, octave=3).midi_num
        if similar_motion and interval_2 in [0, 7, 12, 19]:
            return True
    return False


def similar_motion_combined_with_leaps(cantus, first_species, leap_threshold=3):
    for a1, a2, b1, b2 in zip(cantus[:-1], cantus[1:], first_species[:-1], first_species[1:]):
        similar_motion = (a1 < a2 and b1 < b2) or (a1 > a2 and b1 > b2)
        leap_1 = a2 - a1
        leap_2 = b2 - b1
        large_leap = leap_1 > leap_threshold or leap_2 > leap_threshold
        if similar_motion and large_leap:
            return True
    return False


def num_contrary_motions(cantus, first_species):
    counter = 0
    for a1, a2, b1, b2 in zip(cantus[:-1], cantus[1:], first_species[:-1], first_species[1:]):
        if a1 > a2 and b1 < b2:
            counter += 1
        elif a1 < a2 and b1 > b2:
            counter += 1
    return counter
        

def is_compatible(cantus, first_species, key, final=False):
    return (
        num_repeats(first_species) <= 1
        and not voices_cross(cantus, first_species)
        and not voices_overlap(cantus, first_species)
        and not intervals_exceed_perfect_twelfth(cantus, first_species, key)
        and num_intervals_over_tenth(cantus, first_species, key) <= 2
        and only_consonances(cantus, first_species, key, final)
        and num_perfect_consonances(cantus, first_species, key) <= 4
        and perfect_followed_by_imperfect(cantus, first_species, key)
        and not more_than_three_imperfects_in_a_row(cantus, first_species)
        and not moves_into_perfect_by_similar_motion(cantus, first_species, key)
        and not similar_motion_combined_with_leaps(cantus, first_species, leap_threshold=3)
    )

In [108]:
def try_to_fill_in_first_species(
    cantus,
    skeleton,
    missing_indices,
    key,
):
    first_species = skeleton.copy()

    # Set first note, last note, and second to last note.
    first_species[0] = random.choice([0, 7])
    first_species[-1] = random.choice([0, 7])
    first_species[-2] = -cantus[-2]
    
    degrees = [first_species[i] for i in [0, -1, -2]]
    min_degree = min(degrees)
    max_degree = max(degrees)
    allowed_min = max_degree - 9
    allowed_max = min_degree + 9

    # Iteratively fill in missing indices.
    for idx in missing_indices:
        candidates = list(range(allowed_min, allowed_max))
        random.shuffle(candidates)
        success = False
        for candidate in candidates:
            first_species[idx] = candidate
            is_valid_first_species = (
                is_valid_subcantus(first_species[:idx+1]) \
                and is_compatible(cantus[:idx+1], first_species[:idx+1], key, final=False)
            )
            if is_valid_first_species:
                success = True
                break
        if not success:
            return False, None

    is_valid_first_species = (
        is_valid_subcantus(first_species) \
        and is_compatible(cantus, first_species, key, final=True) \
        and num_contrary_motions(cantus, first_species) >= len(cantus) * 2 // 5
    )
    return is_valid_first_species, first_species


def create_first_species_skeleton(length, climax_idx, climax_degree):
    skeleton = [None] * length
    skeleton[climax_idx] = climax_degree
    return skeleton


def generate_first_species(cantus, length, key):
    """
    Generate a first species counterpoint below a cantus firmus.
    
    TODO: Add code for generating a counterpoint above the cantus.
    """
    skeleton = [None] * length
    missing_indices = [
        i for i in range(length)
        if i != 0 and i != length - 1
    ]

    success = False
    while not success:
        success, first_species = try_to_fill_in_first_species(
            cantus,
            skeleton,
            missing_indices,
            key,
        )
    return first_species

In [109]:
first_species = generate_first_species(
    cantus=cantus,
    length=LENGTH,
    key=KEY,
)

In [110]:
cantus_melody = arrange_melody(
    degrees=cantus,
    rhythm=rhythm,
    octave=4,
    key=KEY,
)
first_species_melody = arrange_melody(
    degrees=first_species,
    rhythm=rhythm,
    octave=3,
    key=KEY,
)

combined_melody = stack_sequences(
    cantus_melody,
    first_species_melody
)
combined_melody_stretched = stretch_note_sequence(
    combined_melody,
    stretch_factor=0.5,
)

note_seq.plot_sequence(combined_melody)
note_seq.play_sequence(combined_melody_stretched, synth=note_seq.fluidsynth)

Loading BokehJS ...

In [318]:
note_seq.sequence_proto_to_midi_file(combined_melody_stretched, '../out/first_species.mid')

Old implementation

```
import itertools


def generate_candidates(num_notes, min_degree, max_degree):
    for candidate in itertools.product(range(min_degree, max_degree), repeat=num_notes):
        yield candidate


def generate_valid_first_halves(num_notes, climax_degree):
    allowed_min_degree = climax_degree - 8
    num_empty_notes = num_notes - 2
    for seq in generate_candidates(num_empty_notes, allowed_min_degree, climax_degree):
        seq = [0] + list(seq) + [climax_degree]
        if (get_range(seq) < 8
            and only_contains_consonant_intervals(seq)
            and not contains_repetitions(seq)
            and max_consecutive_leaps(seq) <= 2
            and not contains_consecutive_leaps_in_same_dir(seq)
            and large_leaps_followed_by_reverse_step(seq)
            and leading_tone_progresses_to_tonic(seq)
            and has_no_leaps_bigger_than_fifth(seq)
            and get_num_big_leaps(seq) <= 1
        ):
            yield seq


def generate_valid_second_halves(num_notes, climax_degree):
    allowed_min_degree = climax_degree - 8
    num_empty_notes = num_notes - 2
    for seq in generate_candidates(num_empty_notes, allowed_min_degree, climax_degree):
        seq = [climax_degree] + list(seq) + [0]
        if (get_range(seq) < 8
            and only_contains_consonant_intervals(seq)
            and not contains_repetitions(seq)
            and max_consecutive_leaps(seq) <= 2
            and not contains_consecutive_leaps_in_same_dir(seq)
            and large_leaps_followed_by_reverse_step(seq)
            and leading_tone_progresses_to_tonic(seq)
            and has_no_leaps_bigger_than_fifth(seq)
            and get_num_big_leaps(seq) <= 1
            and approaches_final_tonic_by_step(seq)
        ):
            yield seq


def generate_cantus_firmus(num_notes, climax_idx, climax_degree):
    first_half_len = climax_idx
    second_half_len = num_notes - climax_idx + 1
    for first_half in generate_valid_first_halves(first_half_len, climax_degree):
        for second_half in generate_valid_second_halves(second_half_len, climax_degree):
            seq = first_half + second_half[1:]
            if (get_range(seq) < 8
                and only_contains_consonant_intervals(seq)
                and not contains_repetitions(seq)
                and max_consecutive_leaps(seq) <= 2
                and not contains_consecutive_leaps_in_same_dir(seq)
                and large_leaps_followed_by_reverse_step(seq)
                and leading_tone_progresses_to_tonic(seq)
                and approaches_final_tonic_by_step(seq)
            ):
                yield seq

cantus_firmus_gen = generate_cantus_firmus(LENGTH, CLIMAX_IDX, CLIMAX_DEGREE)
all_cantus_firmi = list(cantus_firmus_gen)
cantus_firmus = random.choice(all_cantus_firmi)
```